In [5]:

from google.colab import drive
drive.mount("/content/drive")

!pip install transformers

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import torch
import librosa
import numpy as np
import soundfile as sf

from transformers import Wav2Vec2ForCTC, AutoProcessor
import torch
model_id = "facebook/mms-1b-all"

processor = AutoProcessor.from_pretrained(model_id)
model = Wav2Vec2ForCTC.from_pretrained(model_id)


TypeError: ignored

In [15]:
from pydub import AudioSegment
import os
import librosa
import soundfile as sf

def convert_acc_to_wav(file_path):
    # Check if file is .aac format
    _, file_extension = os.path.splitext(file_path)
    if file_extension.lower() != '.aac':
        print(f'Skipped {file_path} (not an .aac file)')
        return

    # Load .acc file
    audio = AudioSegment.from_file(file_path, format="aac")

    # Convert to .wav
    wav_file_path = file_path.rsplit('.', 1)[0] + '.wav'
    audio.export(wav_file_path, format="wav")

    print(f'Successfully converted {file_path} to {wav_file_path}')

    # Load .wav file
    y, sr = librosa.load(wav_file_path, sr=None)

    # Resample to 16000 Hz
    y_16k = librosa.resample(y, orig_sr=sr, target_sr=16000)

    # Save the new .wav file
    new_wav_file_path = wav_file_path.rsplit('.', 1)[0] + '_16kHz.wav'
    sf.write(new_wav_file_path, y_16k, 16000)

    print(f'Successfully converted {wav_file_path} to 16kHz sample rate at {new_wav_file_path}')
    return new_wav_file_path


# Usage
path = convert_acc_to_wav('/content/drive/MyDrive/FYP/ACR/Eng2/table.aac')


Successfully converted /content/drive/MyDrive/FYP/ACR/Eng2/table.aac to /content/drive/MyDrive/FYP/ACR/Eng2/table.wav
Successfully converted /content/drive/MyDrive/FYP/ACR/Eng2/table.wav to 16kHz sample rate at /content/drive/MyDrive/FYP/ACR/Eng2/table_16kHz.wav


In [16]:
import wave

def check_sample_rate(file_path):
    # Open .wav file
    with wave.open(file_path, 'rb') as wav_file:
        # Get sample rate
        sample_rate = wav_file.getframerate()
        print(f'The sample rate of {file_path} is {sample_rate} Hz')

# Usage
check_sample_rate(path)



The sample rate of /content/drive/MyDrive/FYP/ACR/Eng2/table_16kHz.wav is 16000 Hz


In [17]:


def transcribe_audio(wav_file_path):
    # Load audio
    audio_input, sample_rate = sf.read(wav_file_path)

    inputs = processor(audio_input, sampling_rate=16_000, return_tensors="pt")

    with torch.no_grad():
      outputs = model(**inputs).logits

    ids = torch.argmax(outputs, dim=-1)[0]
    transcription = processor.decode(ids)

    return transcription

# Usage:
transcription = transcribe_audio(path)


print(transcription)

what is on the table
